In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU, Bidirectional
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
from alpha_vantage.timeseries import TimeSeries
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

class EnhancedStockPredictor:
    
    def __init__(self, ticker, api_key, sequence_length=120, test_size=0.15):
        self.ticker = ticker
        self.api_key = api_key
        self.data = None
        self.sequence_length = sequence_length  # Increased for more context
        self.test_size = test_size  # Reduced for more training data
        self.scaler = RobustScaler()  # More robust to outliers
        self.model = None
        self.scaled_data = None
        self.features = None

    def fetch_data(self, period="5y"):
        """Enhanced data fetching with more features"""
        print(f"Attempting to fetch data for {self.ticker} using yfinance...")

        try:
            self.data = yf.download(self.ticker, period=period, threads=False)
            if self.data.empty:
                raise ValueError("yfinance returned empty data.")
            print("Data fetched from yfinance.")
        except Exception as e:
            print(f"yfinance failed: {e}")
            print(f"Falling back to Alpha Vantage for {self.ticker}...")
            self.data = self._fetch_from_alpha_vantage(period)
            print("Data fetched from Alpha Vantage.")

        # Enhanced technical indicators
        self.data['MA_5'] = self.data['Close'].rolling(window=5).mean()
        self.data['MA_10'] = self.data['Close'].rolling(window=10).mean()
        self.data['MA_20'] = self.data['Close'].rolling(window=20).mean()
        self.data['MA_50'] = self.data['Close'].rolling(window=50).mean()
        self.data['MA_200'] = self.data['Close'].rolling(window=200).mean()
        
        # More technical indicators
        self.data['RSI'] = self._calculate_rsi(self.data['Close'])
        self.data['MACD'], self.data['MACD_signal'] = self._calculate_macd(self.data['Close'])
        self.data['BB_upper'], self.data['BB_lower'] = self._calculate_bollinger_bands(self.data['Close'])
        self.data['ATR'] = self._calculate_atr(self.data['High'], self.data['Low'], self.data['Close'])
        
        # Price-based features
        self.data['Price_Change'] = self.data['Close'].pct_change()
        self.data['High_Low_Ratio'] = self.data['High'] / self.data['Low']
        self.data['Volume_MA'] = self.data['Volume'].rolling(window=20).mean()
        self.data['Volume_Ratio'] = self.data['Volume'] / self.data['Volume_MA']
        
        # Lag features
        for lag in [1, 2, 3, 5]:
            self.data[f'Close_lag_{lag}'] = self.data['Close'].shift(lag)
            self.data[f'Volume_lag_{lag}'] = self.data['Volume'].shift(lag)
        
        # Clean up
        self.data = self.data.dropna()
        print(f"Final data shape: {self.data.shape}")

    def _fetch_from_alpha_vantage(self, period):
        ts = TimeSeries(key=self.api_key, output_format='pandas')
        outputsize = 'full' if period in ['2y', '5y', 'max'] else 'compact'
        
        try:
            data, _ = ts.get_daily(symbol=self.ticker, outputsize=outputsize)
            data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            data.index = pd.to_datetime(data.index)
            data = data.sort_index()
        except Exception as e:
            raise ValueError(f"Alpha Vantage failed: {e}")
        
        if data.empty:
            raise ValueError(f"No data found for {self.ticker} from Alpha Vantage.")
        return data

    def _calculate_rsi(self, series, period=14):
        delta = series.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))
    
    def _calculate_macd(self, series, fast=12, slow=26, signal=9):
        ema_fast = series.ewm(span=fast).mean()
        ema_slow = series.ewm(span=slow).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=signal).mean()
        return macd, macd_signal
    
    def _calculate_bollinger_bands(self, series, window=20, std_dev=2):
        rolling_mean = series.rolling(window=window).mean()
        rolling_std = series.rolling(window=window).std()
        upper_band = rolling_mean + (rolling_std * std_dev)
        lower_band = rolling_mean - (rolling_std * std_dev)
        return upper_band, lower_band
    
    def _calculate_atr(self, high, low, close, period=14):
        high_low = high - low
        high_close = np.abs(high - close.shift())
        low_close = np.abs(low - close.shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        return true_range.rolling(period).mean()
    
    def prepare_data(self, features=None):
        """Enhanced data preparation with more features"""
        if features is None:
            features = [
                'Close', 'Open', 'High', 'Low', 'Volume',
                'MA_5', 'MA_10', 'MA_20', 'MA_50', 'MA_200',
                'RSI', 'MACD', 'MACD_signal', 'BB_upper', 'BB_lower', 'ATR',
                'Price_Change', 'High_Low_Ratio', 'Volume_Ratio',
                'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_5',
                'Volume_lag_1', 'Volume_lag_2', 'Volume_lag_3', 'Volume_lag_5'
            ]
        
        print("Preparing enhanced data...")
        
        # Select available features (some might not exist if data is limited)
        available_features = [f for f in features if f in self.data.columns]
        print(f"Using {len(available_features)} features: {available_features}")
        
        # Store features for later use
        self.features = available_features
        
        # Select features
        feature_data = self.data[available_features].values
        
        # Scale the data
        self.scaled_data = self.scaler.fit_transform(feature_data)
        
        # Create sequences
        X, y = [], []
        for i in range(self.sequence_length, len(self.scaled_data)):
            X.append(self.scaled_data[i-self.sequence_length:i])
            y.append(self.scaled_data[i, 0])  # Predict 'Close' price (first feature)
            
        X, y = np.array(X), np.array(y)
        
        # Split data
        split_idx = int(len(X) * (1 - self.test_size))
        self.X_train = X[:split_idx]
        self.X_test = X[split_idx:]
        self.y_train = y[:split_idx]
        self.y_test = y[split_idx:]
        
        print(f"Training data shape: X={self.X_train.shape}, y={self.y_train.shape}")
        print(f"Testing data shape: X={self.X_test.shape}, y={self.y_test.shape}")
        
    def build_enhanced_model(self, model_type='lstm_advanced'):
        """Build enhanced model architectures"""
        print(f"Building enhanced model: {model_type}")
        
        self.model = Sequential()
        
        if model_type == 'lstm_advanced':
            # Advanced LSTM with Bidirectional layers
            self.model.add(Bidirectional(LSTM(128, return_sequences=True), 
                                       input_shape=(self.X_train.shape[1], self.X_train.shape[2])))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.2))
            
            self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.2))
            
            self.model.add(LSTM(32, return_sequences=False))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.2))
            
        elif model_type == 'gru_ensemble':
            # GRU-based architecture
            self.model.add(GRU(128, return_sequences=True, 
                              input_shape=(self.X_train.shape[1], self.X_train.shape[2])))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.3))
            
            self.model.add(GRU(64, return_sequences=True))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.3))
            
            self.model.add(GRU(32))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(0.3))
        
        # Dense layers with regularization
        self.model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.3))
        
        self.model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.2))
        
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dense(1, activation='linear'))
        
        # Compile with different optimizers and loss functions
        self.model.compile(
            optimizer=Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999),
            loss='huber',  # More robust to outliers than MSE
            metrics=['mae', 'mse']
        )
        
        print("Enhanced model built successfully!")
        print(self.model.summary())
        
    def train_enhanced_model(self, epochs=200, batch_size=16, validation_split=0.15):
        """Enhanced training with better callbacks"""
        print("Training enhanced model...")
        
        # Enhanced callbacks
        early_stopping = EarlyStopping(
            monitor='val_loss', 
            patience=25,  # Increased patience
            restore_best_weights=True,
            min_delta=1e-6
        )
        
        model_checkpoint = ModelCheckpoint(
            f'{self.ticker}_best_enhanced_model.keras', 
            monitor='val_loss', 
            save_best_only=True,
            save_weights_only=False
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        )
        
        # Train model
        history = self.model.fit(
            self.X_train, self.y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=[early_stopping, model_checkpoint, reduce_lr],
            verbose=1,
            shuffle=True
        )
        
        print("Enhanced model training completed!")
        return history
    
    def evaluate_model(self):
        """Enhanced evaluation with more metrics"""
        print("Evaluating enhanced model...")
        
        # Make predictions
        train_predictions = self.model.predict(self.X_train, verbose=0)
        test_predictions = self.model.predict(self.X_test, verbose=0)
        
        # Inverse transform predictions (more robust method)
        def inverse_transform_predictions(predictions, original_shape):
            # Create dummy array with same shape as original features
            dummy_features = np.zeros((len(predictions), original_shape))
            dummy_features[:, 0] = predictions.flatten()
            # Inverse transform and return only the first column (Close price)
            return self.scaler.inverse_transform(dummy_features)[:, 0]
        
        train_pred_original = inverse_transform_predictions(train_predictions, self.scaled_data.shape[1])
        test_pred_original = inverse_transform_predictions(test_predictions, self.scaled_data.shape[1])
        
        # Inverse transform actual values
        train_actual_original = inverse_transform_predictions(self.y_train.reshape(-1, 1), self.scaled_data.shape[1])
        test_actual_original = inverse_transform_predictions(self.y_test.reshape(-1, 1), self.scaled_data.shape[1])
        
        # Calculate comprehensive metrics
        train_rmse = np.sqrt(mean_squared_error(train_actual_original, train_pred_original))
        test_rmse = np.sqrt(mean_squared_error(test_actual_original, test_pred_original))
        train_mae = mean_absolute_error(train_actual_original, train_pred_original)
        test_mae = mean_absolute_error(test_actual_original, test_pred_original)
        train_r2 = r2_score(train_actual_original, train_pred_original)
        test_r2 = r2_score(test_actual_original, test_pred_original)
        
        # Additional metrics
        train_mape = np.mean(np.abs((train_actual_original - train_pred_original) / train_actual_original)) * 100
        test_mape = np.mean(np.abs((test_actual_original - test_pred_original) / test_actual_original)) * 100
        
        metrics = {
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_mae': train_mae,
            'test_mae': test_mae,
            'train_r2': train_r2,
            'test_r2': test_r2,
            'train_mape': train_mape,
            'test_mape': test_mape
        }
        
        print("Enhanced Model Evaluation Metrics:")
        print(f"Training RMSE: ${train_rmse:.4f}")
        print(f"Testing RMSE: ${test_rmse:.4f}")
        print(f"Training MAE: ${train_mae:.4f}")
        print(f"Testing MAE: ${test_mae:.4f}")
        print(f"Training R²: {train_r2:.4f}")
        print(f"Testing R²: {test_r2:.4f}")
        print(f"Training MAPE: {train_mape:.2f}%")
        print(f"Testing MAPE: {test_mape:.2f}%")
        
        return metrics
    
    def predict_future(self, days=10):
        """
        Predict future stock prices for the specified number of days
        """
        print(f"Predicting future prices for {days} days...")
        
        # Get the last sequence_length days of scaled data
        last_sequence = self.scaled_data[-self.sequence_length:].copy()
        future_predictions = []
        
        for day in range(days):
            # Reshape for prediction
            current_sequence = last_sequence.reshape(1, self.sequence_length, len(self.features))
            
            # Make prediction
            next_pred = self.model.predict(current_sequence, verbose=0)[0, 0]
            future_predictions.append(next_pred)
            
            # Update the sequence for next prediction
            # Create new row with predicted price and estimated other features
            new_row = last_sequence[-1].copy()  # Copy last row
            new_row[0] = next_pred  # Update the Close price (first feature)
            
            # For simplicity, we'll use the last known values for other features
            # In a more sophisticated approach, you might predict these as well
            
            # Shift the sequence and add the new prediction
            last_sequence = np.roll(last_sequence, -1, axis=0)
            last_sequence[-1] = new_row
        
        # Inverse transform predictions to get actual prices
        future_predictions = np.array(future_predictions).reshape(-1, 1)
        
        # Create dummy array for inverse transform
        dummy_features = np.zeros((len(future_predictions), len(self.features)))
        dummy_features[:, 0] = future_predictions.flatten()
        
        # Inverse transform to get actual prices
        future_prices = self.scaler.inverse_transform(dummy_features)[:, 0]
        
        return future_prices
    
    def save_model(self):
        """Save the trained model and scaler"""
        model_filename = f'{self.ticker}_enhanced_model.keras'
        scaler_filename = f'{self.ticker}_scaler.pkl'
        
        # Save model
        self.model.save(model_filename)
        print(f"Model saved as {model_filename}")
        
        # Save scaler
        joblib.dump(self.scaler, scaler_filename)
        print(f"Scaler saved as {scaler_filename}")
        
        # Save additional metadata
        metadata = {
            'features': self.features,
            'sequence_length': self.sequence_length,
            'ticker': self.ticker
        }
        
        metadata_filename = f'{self.ticker}_metadata.pkl'
        joblib.dump(metadata, metadata_filename)
        print(f"Metadata saved as {metadata_filename}")
    
    def load_model(self, model_filename=None, scaler_filename=None, metadata_filename=None):
        """Load a previously trained model"""
        if model_filename is None:
            model_filename = f'{self.ticker}_enhanced_model.keras'
        if scaler_filename is None:
            scaler_filename = f'{self.ticker}_scaler.pkl'
        if metadata_filename is None:
            metadata_filename = f'{self.ticker}_metadata.pkl'
        
        # Load model
        self.model = load_model(model_filename)
        print(f"Model loaded from {model_filename}")
        
        # Load scaler
        self.scaler = joblib.load(scaler_filename)
        print(f"Scaler loaded from {scaler_filename}")
        
        # Load metadata
        metadata = joblib.load(metadata_filename)
        self.features = metadata['features']
        self.sequence_length = metadata['sequence_length']
        print(f"Metadata loaded from {metadata_filename}")

# Enhanced training function
def train_enhanced_stock_model(ticker_symbol, api_key, model_type='lstm_advanced', save_model=True):
    """
    Enhanced training pipeline for better MAE
    """
    # Initialize enhanced predictor
    predictor = EnhancedStockPredictor(ticker_symbol, api_key, sequence_length=120, test_size=0.15)
    
    # Fetch and prepare enhanced data
    predictor.fetch_data(period="5y")  # More data
    predictor.prepare_data()
    
    # Build and train enhanced model
    predictor.build_enhanced_model(model_type=model_type)
    history = predictor.train_enhanced_model(epochs=200, batch_size=16)
    
    # Evaluate model
    metrics = predictor.evaluate_model()
    
    # Make future predictions
    future_predictions = predictor.predict_future(days=10)
    
    print("\nFuture Predictions (Next 10 days):")
    for i, pred in enumerate(future_predictions, 1):
        print(f"Day {i}: ${pred:.2f}")
    
    # Save model if requested
    if save_model:
        predictor.save_model()
    
    return predictor, history, metrics, future_predictions

if __name__ == "__main__":
    # Set your API key
    API_KEY = "JO6S4PKBXUZMZWE1"
    
    # Example: Train enhanced model for Apple stock
    ticker = "AAPL"
    print("Training Enhanced Stock Predictor...")
    print("This will take longer but should achieve better MAE!")
    
    predictor, history, metrics, predictions = train_enhanced_stock_model(
        ticker, API_KEY, model_type='lstm_advanced', save_model=True
    )
    
    print(f"\nEnhanced training completed for {ticker}")
    print(f"Testing MAE: ${metrics['test_mae']:.4f}")
    print("Model files saved and ready for use!")

Training Enhanced Stock Predictor...
This will take longer but should achieve better MAE!
Attempting to fetch data for AAPL using yfinance...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


yfinance failed: yfinance returned empty data.
Falling back to Alpha Vantage for AAPL...
Data fetched from Alpha Vantage.
Final data shape: (6242, 28)
Preparing enhanced data...
Using 27 features: ['Close', 'Open', 'High', 'Low', 'Volume', 'MA_5', 'MA_10', 'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'MACD_signal', 'BB_upper', 'BB_lower', 'ATR', 'Price_Change', 'High_Low_Ratio', 'Volume_Ratio', 'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_5', 'Volume_lag_1', 'Volume_lag_2', 'Volume_lag_3', 'Volume_lag_5']
Training data shape: X=(5203, 120, 27), y=(5203,)
Testing data shape: X=(919, 120, 27), y=(919,)
Building enhanced model: lstm_advanced
Enhanced model built successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 120, 256)       │       159,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 120, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 120, 128)       │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 120, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 120, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 351,489 (1.34 MB)

 Trainable params: 350,465 (1.34 MB)

 Non-trainable params: 1,024 (4.00 KB)

None
Training enhanced model...
Epoch 1/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 31s 85ms/step - loss: 6.0664 - mae: 0.8787 - mse: 1.3319 - val_loss: 4.3259 - val_mae: 0.3639 - val_mse: 0.1943 - learning_rate: 5.0000e-04
Epoch 2/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - loss: 3.9901 - mae: 0.4887 - mse: 0.4473 - val_loss: 2.6178 - val_mae: 0.2916 - val_mse: 0.1258 - learning_rate: 5.0000e-04
Epoch 3/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - loss: 2.3456 - mae: 0.4004 - mse: 0.3102 - val_loss: 1.3577 - val_mae: 0.2847 - val_mse: 0.1505 - learning_rate: 5.0000e-04
Epoch 4/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - loss: 1.1629 - mae: 0.3378 - mse: 0.2286 - val_loss: 0.5848 - val_mae: 0.2610 - val_mse: 0.1119 - learning_rate: 5.0000e-04
Epoch 5/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - loss: 0.5151 - mae: 0.3036 - mse: 0.1863 - val_loss: 0.2701 - val_mae: 0.2423 - val_mse: 0.1232 - learning_rate: 5.0000e-04
Epoch 6/200
277/277 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - loss: 

AttributeError: 'EnhancedStockPredictor' object has no attribute 'predict_future'

###
Training Enhanced Stock Predictor...
###
This will take longer but should achieve better MAE!
Attempting to fetch data for AAPL using yfinance...
YF.download() has changed argument auto_adjust default to True
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
yfinance failed: yfinance returned empty data.
Falling back to Alpha Vantage for AAPL...
Data fetched from Alpha Vantage.
Final data shape: (6242, 28)
Preparing enhanced data...
Using 27 features: ['Close', 'Open', 'High', 'Low', 'Volume', 'MA_5', 'MA_10', 'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'MACD_signal', 'BB_upper', 'BB_lower', 'ATR', 'Price_Change', 'High_Low_Ratio', 'Volume_Ratio', 'Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_5', 'Volume_lag_1', 'Volume_lag_2', 'Volume_lag_3', 'Volume_lag_5']
Training data shape: X=(5203, 120, 27), y=(5203,)
Testing data shape: X=(919, 120, 27), y=(919,)
Building enhanced model: lstm_advanced
Enhanced model built successfully!